<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>VantageCloud Lake Systems Scaling and Monitoring</b></p>
<hr>

<br>

<b style = 'font-size:24px;font-family:Arial;color:#E37C4D'>Demo 2 - Real-Time monitoring of Key Metrics</b>

<p style = 'font-size:16px;font-family:Arial'>This notebook is a short set of python code designed to illustrate how key metrics such as compute cluster status, active user sessions, and CPU utilization can be monitored using dashboards or other monitoring tools</p>

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Demonstration Overview</b>

<p style = 'font-size:16px;font-family:Arial'>This notebook consists of three primary demonstrations</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li><b style = 'color:#00b2b1'>Connect to the VantageCloud Lake System</b> - Connect as a user with access to the metrics service and performance monitoring functions</li>
    <li><b style = 'color:#00b2b1'>Key Metrics Queries</b> - Queries that monitor active users, Cluster CPU stats, and number of instances</li>
    <li><b style = 'color:#00b2b1'>Dasbhboard</b> - Update and plot stats every three seconds</li>
    </ol>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'>Python Package Imports</p>

<p style = 'font-size:16px;font-family:Arial'>
Import the required packages including teradata client packages (teradataml and teradatasql) as well as packages for plotting and data management.</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import teradatasql, logging, time, math, json
from teradataml import *
import getpass
import datetime
import pandas as pd
import numpy as np

from time import sleep
from random import random
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
%matplotlib inline

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'>Connect to Vantage</p>

<p style = 'font-size:16px;font-family:Arial'>Before performing any operations in Vantage, we need to connect to the system.  The below code will read in a variables file (vars.json - this has been used in prior environment setup and data engineering examples) and will connect to Vantage with this information.  The Vantage connection is referred to as a "Context" - a common python-rdbms connection architecture.</p> 

In [ ]:
# create a local dictionary of environment-specific variables

# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

# Use the "data_engineer" and Business compute group from the base setup
host = session_vars['environment']['host']
username = session_vars['hierarchy']['users']['business_users'][0]['username']
password = session_vars['hierarchy']['users']['business_users'][0]['password']
compute_group = session_vars['hierarchy']['users']['business_users'][0]['compute_group']

conn_info = {}
conn_info['host'] = host
conn_info['username'] = username
conn_info['password'] = password
conn_info['compute_group'] = compute_group

eng = create_context(host = conn_info['host'], username = conn_info['username'], password = conn_info['password'])

# confirm connection
print(eng)

<hr>
<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Key Metrics Queries</b></p>



<p style = 'font-size:16px;font-family:Arial'>
Execute initial queries to collect Compute Profile status, Active user sessions, and Physical CPU percentage utilization.  More information on these and other queries are found in Demonstration 3.</p>

In [ ]:

# Query to get compute profile status - Active, Hibernated, etc.
clusters_query = '''
SELECT * FROM dbc.computestatusv
ORDER BY ComputeProfileName, CurrentState;
'''

# AmpState "Active" is a good way to see in-flight queries 
cons_query = '''
SELECT HostId (TITLE 'Host',FORMAT 'ZZZ9'),
    UserName (TITLE 'Active Users'),
    SessionNo (TITLE 'Session'),
 LogonPENo (TITLE 'IFP')
FROM TABLE (SYSLIB.MonitorSession(-1,'*',0)) AS t1
WHERE AmpState='Active'
ORDER BY 1,2;
'''

# A simple query to collect CPU Utilization on physical resources
all_cpus_query = '''
SELECT 
CASE
WHEN Name ='1' then 'Primary Cluster'
ELSE Name
END as Name, AVG(CPUUse) as CPUUse
FROM syslib.MonitorPhysicalResourceSV(USING details('1')) AS dt
GROUP BY 1;
'''



try:
    df_concurrency = pd.read_sql(cons_query, eng)
except Exception as e:
        if str(e.args).find("Error 7559") >= 1: # check for occasional malloc issue
            df_concurrency = pd.read_sql(cons_query, eng)
            pass
        else:
            raise

df_all_cpus = pd.DataFrame()           
df_cpu = pd.read_sql(all_cpus_query, eng)
df_cpu['id'] = 0
df_all_cpus = df_all_cpus.append(df_cpu.pivot(index = 'id', columns = 'Name', values = 'CPUUse'))

df_clusters = pd.read_sql(clusters_query, eng)
df_cluster_count = df_clusters[df_clusters['CurrentState'] == 'Active'].groupby('ComputeProfileName').count()[['InstanceName']]
df_cluster_count['id'] = 0
df_all_clusters = df_cluster_count.reset_index().pivot(index = 'id', columns = 'ComputeProfileName', values = 'InstanceName')

plt.rcParams["figure.figsize"] = (20,5)

x = np.array([[0, df_concurrency.shape[0]]])
p = 0

<hr>
<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Dashboard</b></p>



<p style = 'font-size:16px;font-family:Arial'>
A simple Python loop using matplotlib plotting tools can be used to simulate a dashboard or other monitoring tool.  To exit this loop, wait the desired iterations, or choose Kernel->Interrupt from the Jupyter menu.</p>

In [ ]:
for i in range(1200):

    p+=1 # keep a persistent value to maintain plot index
    fig, (ax1, ax2, ax3) = plt.subplots(ncols = 3)

    try:
        df_concurrency = pd.read_sql(cons_query, eng)
    except Exception as e:
        if str(e.args).find("Error 7559") >= 1: # check for occasional malloc issue
            pass
        else:
            raise
    
    df_cpu = pd.read_sql(all_cpus_query, eng)
    df_cpu['id'] = p
    df_all_cpus = df_all_cpus.append(df_cpu.pivot(index = 'id', columns = 'Name', values = 'CPUUse'))

    df_clusters = pd.read_sql(clusters_query, eng)
    df_cluster_count = df_clusters[df_clusters['CurrentState'] == 'ACTIVE'].groupby('ComputeProfileName').count()[['InstanceName']]
    df_cluster_count['id'] = p
    df_all_clusters = df_all_clusters.append(df_cluster_count.reset_index().pivot(index = 'id', columns = 'ComputeProfileName', values = 'InstanceName'))

    x = np.append(x, [[p,df_concurrency.shape[0]]], axis = 0)

    clear_output(wait=True)
    sleep(3)

    ax1.plot(x[:, 0], x[:, 1])
    ax1.set_title('Concurrency')

    df_all_cpus.sort_index().plot(kind = 'line', ax = ax2)
    ax2.set_title('CPU Utilization')

    df_all_clusters.fillna(0).sort_index().plot(kind = 'line', ax = ax3)
    ax3.set_title('Instance Count')

    plt.show()

    display(df_clusters)
    display(df_concurrency)


Copyright 2023, Teradata Corporation